In [2]:
import numpy as np
from pandas import read_csv
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
np.set_printoptions(precision=3, suppress=True) 
from datetime import datetime
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

###### Reference for preprocessing ideas
###### ref : http://cs229.stanford.edu/proj2014/Tanner%20Gilligan,%20Jean%20Kono,%20Prediction%20of%20Bike%20Rentals.pdf
###### ref : https://towardsdatascience.com/predicting-no-of-bike-share-users-machine-learning-data-visualization-project-using-r-71bc1b9a7495

In [64]:
#Dropping not predictive attributes : instant
data = np.loadtxt('hour.csv', delimiter=',', skiprows=1, usecols=(range(2,17)))
data2 = np.loadtxt('hour.csv', delimiter=',', skiprows=1, usecols=(1), dtype='|S10').astype(str)
X = data[:,:-1]
y = data[:,-1]

# Preprocessing step: extracting week number from date attribute
for i in range(data2.shape[0]):
    data2[i] = datetime.date(datetime.strptime(data2[i], '%Y-%m-%d')).isocalendar()[1]
data2 = data2.astype(int)
print(data2)
data = np.column_stack((data,data2.reshape(-1,1)))
print(data.shape)


[52 52 52 ...  1  1  1]
(17379, 16)


In [11]:
class Bike_Sharing:
    
    def __init__(self):
        return
    
    # reading data 
    def read_data(self):        
       #Dropping not predictive attributes : instant
        data = np.loadtxt('hour.csv', delimiter=',', skiprows=1, usecols=(range(2,17)))
        data2 = np.loadtxt('hour.csv', delimiter=',', skiprows=1, usecols=(1), dtype='|S10').astype(str)
        
        # Preprocessing step: extracting week number from date attribute
        for i in range(data2.shape[0]):
            data2[i] = datetime.date(datetime.strptime(data2[i], '%Y-%m-%d')).isocalendar()[1]
        data2 = data2.astype(int)        
        data = np.column_stack((data2.reshape(-1,1), data))
        return data
    
    def preprocessing(self, data):               
        # splitting data
        X = data[:,:-1]
        y = data[:,-1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)        
        #preprocessing using standard scaler
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        return X_train, y_train, X_test, y_test
    
    def dim_Reduction(self, X_train, X_test):
        # Dimensionality Reduction using PCA from 123 dims to 20 dims
        pca = PCA(n_components = 20)
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)
        return X_train, X_test
        
    def cv_SVR(self, X, y):
        #scorer = make_scorer(neg_mean_squared_error)
        C_grid = [0.1, 1, 10]
        gamma_grid = np.logspace(-2, 1, 4)[0:3]
        svm = sklearn.svm.SVR(kernel='rbf')
        param_grid = { 'C' : C_grid, 'gamma' : gamma_grid, 'kernel' : ['rbf', 'sigmoid',  'linear']}
        gridcv = sklearn.model_selection.GridSearchCV(svm, param_grid, n_jobs=-1, verbose=1, cv=3)
        #, scoring = 'neg_mean_squared_error'
        gridcv.fit(X_train, y_train)
        print("best parameters:", gridcv.best_params_)
        print("%.1f%% neg mean squared error on validation sets (average)" % (gridcv.best_score_*100))
        return gridcv.best_params_
    
    def cv_DTR(self, X, y):
        dt = DecisionTreeRegressor()
        param_grid = {
            "min_samples_split" : np.random.random_sample((100,)),
            "min_samples_leaf" : np.arange(1,6),
            'max_depth': range(1, 20),
            'criterion' : ['mse', 'mae', 'friedman_mse'],
            'splitter' : ['best', 'random'],
        }
        return Bike_Sharing.randomCV(dt, X, y, param_grid, 50, 6)
        
    def cv_RandomForest(self, X, y):
        rf = RandomForestRegressor()
        param_grid = {
            #"n_estimators" : [10*x for x in np.arange(1,25)],
            "min_samples_split" : np.random.random_sample((100,)),
            "min_samples_leaf" : np.arange(1,6),
            'max_depth': range(1, 20),
        }
        return Bike_Sharing.randomCV(rf, X, y, param_grid, 40, 6)
        
    def cv_adaBoost(self, X, y):
        #scorer = make_scorer(precision_score)
        ada_boost = AdaBoostRegressor(n_estimators=50, learning_rate=1)
        param_grid = {'n_estimators': range(1, 50), 'learning_rate': [0.1, 0.5, 1]}
        gridcv = sklearn.model_selection.GridSearchCV(ada_boost, param_grid, verbose=1, cv=3, n_jobs=-1)
                                                      #, scoring='explained_variance')
        gridcv.fit(X, y)
        print("best parameters:", gridcv.best_params_)
        print("%.1f%% validation on validation sets (average)" % (gridcv.best_score_))
        return gridcv.best_params_
    
    def cv_linReg(self, X, y):
        lr = LinearRegression()
        param_grid = {
            "fit_intercept" : [True, False],
        }
        return Bike_Sharing.randomCV(lr, X, y, param_grid, 40, 6)
        
    def cv_GP(self, X, y):
        clf = GaussianProcessRegressor()
        param_grid = {
            
        "normalize_y" : [True, False],
        "copy_X_train" : [True, False],
        "alpha" : np.linspace(0, 5, 100),
        }
        return Bike_Sharing.randomCV(clf, X, y, param_grid, 10, 6)
    
    def cv_NNRegressor(self, X, y):
        nn = sklearn.neural_network.MLPRegressor()

        param_grid ={
                    'hidden_layer_sizes' : range(2,100),
                    "activation" : ['identity', 'logistic', 'tanh', 'relu']
                    }
        return Bike_Sharing.randomCV(nn, X, y, param_grid, 50, 5)
        
    def randomCV(clf, X, y, param_grid, n_iter, cv):
        #scorer = make_scorer(precision_score)
        random_search = RandomizedSearchCV(clf, param_distributions = param_grid, n_iter = n_iter, cv = cv, iid = False, 
                                           verbose=1, n_jobs=-1, scoring='explained_variance')
        #scoring = "explained_variance"
        random_search.fit(X, y)
        #print(random_search.cv_results_)
        Bike_Sharing.report(random_search.cv_results_)
        return random_search.best_params_
    
    def report(results, n_top=1):
        for i in range(1, n_top + 1):
            candidates = np.flatnonzero(results['rank_test_score'] == i)
            k = 0
            for candidate in candidates:                
                print("Model with rank: {0}".format(i))
                print("Variance on validation data: {0:.3f} (std: {1:.3f})".format(
                      results['mean_test_score'][candidate],
                      results['std_test_score'][candidate]))
                print("Parameters: {0}".format(results['params'][candidate]))
                print("")
                k += 1
                if k == 3:
                    break
                
    def predict(self, model, X_test, y_test):
        predict = model.predict(X_test)
        predict[predict<0] =0
        rmse = mean_squared_error(y_test, predict)
        print("MSE on test data : ", rmse)

In [137]:
np.where(np.isnan(data))
y_train[y_train<0]

array([], dtype=float64)

In [9]:
if __name__ == "__main__":
    obj = Bike_Sharing()
    data = obj.read_data()
    X_train, y_train, X_test, y_test = obj.preprocessing(data)
    print('---------SVR--------')
    model = obj.cv_SVR(X_train, y_train)
    reg = sklearn.svm.SVR().set_params(**model).fit(X_train, y_train)
    obj.predict(reg, X_test, y_test)
    print('---------DTR--------')
    model = obj.cv_DTR(X_train, y_train)
    reg = sklearn.tree.DecisionTreeRegressor().set_params(**model).fit(X_train, y_train)
    obj.predict(reg, X_test, y_test)
    print('---------Random Forrest Regressor--------') 
    model = obj.cv_RandomForest(X_train, y_train)
    reg = sklearn.ensemble.RandomForestRegressor().set_params(**model).fit(X_train, y_train)
    obj.predict(reg, X_test, y_test)
    print('---------Adaboost Regressor--------')
    model = obj.cv_adaBoost(X_train, y_train)
    reg = sklearn.ensemble.AdaBoostRegressor().set_params(**model).fit(X_train, y_train)
    obj.predict(reg, X_test, y_test)
    print('---------Gaussian Process Regressor--------')
    model = obj.cv_GP(X_train, y_train)
    reg = sklearn.gaussian_process.GaussianProcessRegressor().set_params(**model).fit(X_train, y_train)
    obj.predict(reg, X_test, y_test)
    print('---------Linear Regressor--------')
    model = obj.cv_linReg(X_train, y_train)
    reg = LinearRegression().set_params(**model).fit(X_train, y_train)
    obj.predict(reg, X_test, y_test)
    print('---------NN Regressor--------')
    model = obj.cv_NNRegressor(X_train, y_train)
    reg = MLPRegressor().set_params(**model).fit(X_train, y_train)
    obj.predict(reg, X_test, y_test)

---------SVR--------
Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  3.3min finished


best parameters: {'C': 10, 'gamma': 0.01, 'kernel': 'linear'}
100.0% neg mean squared error on validation sets (average)
MSE on test data :  0.0026928840122218403
---------DTR--------
Fitting 6 folds for each of 50 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.0min finished


Model with rank: 1
Variance on validation data: 0.999 (std: 0.000)
Parameters: {'splitter': 'best', 'min_samples_split': 0.0017179005601895003, 'min_samples_leaf': 4, 'max_depth': 19, 'criterion': 'friedman_mse'}

MSE on test data :  48.784290385617545
---------Random Forrest Regressor--------
Fitting 6 folds for each of 40 candidates, totalling 240 fits


...

Model with rank: 1
Variance on validation data: 0.969 (std: 0.003)
Parameters: {'min_samples_split': 0.050211853128626305, 'min_samples_leaf': 2, 'max_depth': 6}

MSE on test data :  909.6210889130911
---------Adaboost Regressor--------
Fitting 3 folds for each of 147 candidates, totalling 441 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 212 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 441 out of 441 | elapsed:  1.3min finished


best parameters: {'learning_rate': 1, 'n_estimators': 36}
1.0% validation on validation sets (average)
MSE on test data :  597.9960944279804
---------Gaussian Process Regressor--------
Fitting 6 folds for each of 25 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


MemoryError: 

In [12]:
print('---------Gaussian Process Regressor--------')
model = obj.cv_GP(X_train, y_train)
reg = sklearn.gaussian_process.GaussianProcessRegressor().set_params(**model).fit(X_train, y_train)
obj.predict(reg, X_test, y_test)
print('---------Linear Regressor--------')
model = obj.cv_linReg(X_train, y_train)
reg = LinearRegression().set_params(**model).fit(X_train, y_train)
obj.predict(reg, X_test, y_test)
print('---------NN Regressor--------')
model = obj.cv_NNRegressor(X_train, y_train)
reg = MLPRegressor().set_params(**model).fit(X_train, y_train)
obj.predict(reg, X_test, y_test)

---------Gaussian Process Regressor--------
Fitting 6 folds for each of 25 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.9min


KeyboardInterrupt: 